Importing the needed packages:

In [ ]:
import scipy.io as sio
import numpy as np
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Activation, Permute, Dropout, Concatenate, Average, Reshape, Multiply
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, AveragePooling1D, Conv1D, MaxPooling1D
from tensorflow.keras.layers import SeparableConv2D, DepthwiseConv2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import SpatialDropout2D
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.layers import Input, Flatten
from tensorflow.keras.constraints import max_norm
from tensorflow.keras import backend as K
import os
import time
from sklearn.metrics import confusion_matrix

from keras.backend import expand_dims
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.constraints import NonNeg
from sklearn.metrics import classification_report
from sklearn import manifold
import matplotlib.pyplot as plt
from keras.models import model_from_json
from tensorflow.keras.utils import plot_model
import matplotlib.patches as mpatches
from umap.parametric_umap import ParametricUMAP
from deepexplain.tensorflow import DeepExplain


The following section implements the hierarchial self-attention mechanism module. This section uses the following implementation:
https://github.com/philipperemy/keras-attention-mechanism

In [ ]:
# Attention_ViewSelector Class:
from tensorflow.keras.layers import Dense, Lambda, dot, Activation, concatenate
from tensorflow.keras.layers import Layer


class Attention_ViewSelector(Layer):

    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def __call__(self, hidden_states):
        """
        Many-to-one attention mechanism for Keras.
        @param hidden_states: 3D tensor with shape (batch_size, time_steps, input_dim).
        @return: 2D tensor with shape (batch_size, 128)
        @author: felixhao28.
        """
        hidden_size = int(hidden_states.shape[2])
        # Inside dense layer
        #              hidden_states            dot               W            =>           score_first_part
        # (batch_size, time_steps, hidden_size) dot (hidden_size, hidden_size) => (batch_size, time_steps, hidden_size)
        # W is the trainable weight matrix of attention Luong's multiplicative style score
        score_first_part = Dense(hidden_size, use_bias=False)(hidden_states)
        #            score_first_part           dot        last_hidden_state     => attention_weights
        # (batch_size, time_steps, hidden_size) dot   (batch_size, hidden_size)  => (batch_size, time_steps)
        h_t = Lambda(lambda x: x[:, -1, :], output_shape=(hidden_size,))(hidden_states)
        score = dot([score_first_part, h_t], [2, 1])
        attention_weights = Activation('softmax')(score)
        # (batch_size, time_steps, hidden_size) dot (batch_size, time_steps) => (batch_size, hidden_size)
        context_vector = dot([hidden_states, attention_weights], [1, 1])
        return context_vector


In the following section, the vector containing the all subjects' numbers must be loaded:

In [ ]:
#%% <<<<<<<<< num_sub_good >>>>>>>>>>>>>>>>>>>>>>>>>>>
num_sub_all = "vector of the subjects' number"
test_num = len(num_sub_all)


In the next section, three data views and the true labels of all epochs of all subjects must be loaded. Note that we have 22 EEG channels (montages) and our EEG epochs have 3 seconds length, which results in 3 \times Fs ($3 \times 250Hz =750$) time samples. The details of our three views are as follows:

1) View1: this EEG view is raw denoised EEG time samples having the size of (None, channels=22, EEG samples=750, 1).

2) View2: this view is the Time-Frequency (Morse scalogram) representation of the corresponding EEG channel signals of EEG epochs, which are resized to (32*32) frames, resulting in the size of (None, 32, 32, EEG channels=22).

3) View3: this view consists of a wide range of the EEG hand-engineered features reported to be efficient in the EEG seizure detection tasks. These features listed in our paper result in 946 features for all channels signals combined (and, therefore, for an EEG epoch) and the total size of (None, 946). 


In [ ]:
#%% <<<<<<<<< MLP NBML features >>>>>>>>>>>>>>>>>>>>>>>>>>>

View1 = 'View1 data: Time-sample EEG signals with size of (None, 22, 750, 1)'

View2 = 'View2 data: Time-Frequency (Scalogram) images of the EEG signals with size of (None, 32, 32, 22)'

View3 = 'View3 data: Hand-engineered features with size of (None, 946)'

Labels = 'Labels of epochs of all subjects'


A folder path for saving the results are specified in the next section.

In [ ]:
path_folder = 'A folder path for save the model weights and results'

The following section implements the Leave-One-Subject-Out (LOSO) scheme for obtaining the results on the unseen subject data. The details are provided as follows:

*   Section 1) Three data views of the out subject and train subjects are loaded. 
*   Section 2-4) Three ViewNet (i.e., ViewNet1-3 in our paper) are designed. These networks process the EEG time samples, TF representations, and hand-engineered features, respectively. Note that the ViewNet1 is inspired by [EEGNET](https://github.com/vlawhern/arl-eegmodels).
*   Section 5) The ViewNet1-3 are merged using the self-attention module and form the proposed fAttNet in our paper.
*   Section 6) Using class weight property, the classification imbalance problem is trying to be addressed.
*   Section 7) The proposed fAttNet is trained on 5 epochs on the training subjects using the Categorical cross-entropy as the loss function and Adam as the optimizer. 
*   Section 8) The weights of the trained fAttNet can be saved in this section. The absolute weights corresponding to the spatial filter, having 22 elements, of the ViewNet1 part of the fAttNet can be averaged over all trained fAttNets on all unseen subjects to show the topography of the **Figure 15** in our paper. Also, the spectrograms of the weights corresponding to the temporal filters of an example trained fAttNet can result in the **Figure 14** in our paper.
*   Section 9) The confusion matrix and different classification metrics (i.e., Accuracy, Sensitivity, Specificity, Precision, and F1-Score) corresponding to the prediction of the trained fAttNet on the unseen subject are obtained can be accumulated with the obtained previous results on the previous unseen subjects to yield the final results on all unseen subjects, which are reported in **Tables 1 and 2** in our paper.
*   Section 10) In this section, the outputs of the final flatten layer of the fAttNet on the unseen subject data are obtained to be ready for the t-SNE and parametric UMAP feature reduction methods.
*   Section 11) Using t-SNE, the obtained embedding in Section 10 is reduced to the 2 dimensions and is visualized to show the discriminability aspect of the proposed fAttNet. These results are shown in **Figure 17** (top row) in our paper.
*   Section 12) Similar to the previous section (i.e., Section 11) but using the parametric UMAP method. Note that for implementing the non-linear embedding in this method, we use a fully connected network with 100 neurons. These results are shown in **Figure 17** (top row) in our paper.
*   Section 13) Finally, to obtain the saliency maps on the unseen data subjects, we have used the [Deep Explain](https://github.com/marcoancona/DeepExplain) library. Using this section, one can obtain the saliency maps corresponding to all EEG epochs. Note that for obtaining the results shown in the **Figure 16** of our paper, we just executed the obtained saliency maps corresponding to the View-Net1 part of the fAttNet.

In [ ]:
for Out_Subject in np.arange(0, len(num_sub_all)):
    
    print('\nSubject_out: ' + str(Out_Subject) + '(' + str(num_sub_all[Out_Subject]) + ')' + '/'+ str(len(num_sub_all))+' started...')
        
    
#%% Section 1: #########################################3###############################################################################################################################:
    View1_test = 'View1 test data: data of the out subject'
    View3_test = 'View3 test data: data of the out subject'
    View2_test = 'View2 test data: data of the out subject'
    labels_test = "labels of the out subject's epochs"

    View1_train = 'View1 train data: data of the train subjects'
    View3_train = 'View2 train data: data of the train subjects'
    View2_train = 'View3 train data: data of the train subjects'
    labels_train = "labels of the train subjects' epochs"   
    
    
#%% Section 2: View-Net1: EEGNET model #########################################3###############################################################################################################################:
    nb_classes = 2
    Chans = 22
    Samples = 750
    dropoutRate = 0.5
    kernLength = 32
    F1 = 4
    D = 1
    F2 = F1*D 
    dropoutType = 'Dropout'
    norm_rate = 0.25
    Concat_Dense = 16
        
    input1   = Input(shape = (Chans, Samples, 1))

    block1       = Conv2D(F1, (1, kernLength), padding = 'same',
                                   input_shape = (Chans, Samples, 1),
                                   use_bias = False)(input1)
    block1       = BatchNormalization(axis = 1)(block1)
    block1       = DepthwiseConv2D((Chans, 1), use_bias = False, 
                                   depth_multiplier = D,
                                   depthwise_constraint = max_norm(1.))(block1)
    block1       = BatchNormalization(axis = 1)(block1)
    block1       = Activation('elu')(block1)
    block1       = AveragePooling2D((1, 4))(block1)
    block1       = Dropout(dropoutRate)(block1)
        
    block2       = SeparableConv2D(F2, (1, 16), use_bias = False, padding = 'same')(block1)
    block2       = BatchNormalization(axis = 1)(block2)
    block2       = Activation('elu')(block2)
    block2       = AveragePooling2D((1, 4))(block2)
    block2       = Dropout(dropoutRate)(block2)
        
    flatten1      = Flatten()(block2)
    flatten1        = Dense(Concat_Dense)(flatten1)
    flatten1_temp = Reshape((1, flatten1.shape[-1]))(flatten1)
    
    dense_model1        = Dense(2, kernel_constraint = max_norm(norm_rate))(flatten1)
    softmax1      = Activation('softmax')(dense_model1)

    model1 = Model(inputs=input1, outputs=softmax1)
    model1_Embedder = Model(inputs=input1, outputs=flatten1)

    softmax1_temp = Reshape((softmax1.shape[-1],1))(softmax1)
    
    
#%% Section 3: View-Net2: CNN-image model ######################################################################################################################################################################3:
        
    input2   = Input(shape = (View2_test.shape[1], View2_test.shape[2], View2_test.shape[3]))

    block1       = BatchNormalization(axis = 1)(input2)
    block1       = Conv2D(32, (7, 7), padding = 'valid',
                                   input_shape = (View2_test.shape[1], View2_test.shape[2], View2_test.shape[3]))(block1)    
    block1       = BatchNormalization(axis = 1)(block1)
    block1       = Activation('elu')(block1)
    block1       = Conv2D(32, (7, 7), padding = 'valid')(block1)
    block1       = BatchNormalization(axis = 1)(block1)
    block1       = Activation('elu')(block1)
    block1       = MaxPooling2D((2, 2))(block1)
    block1       = Dropout(dropoutRate)(block1)
    
    block2       = Conv2D(32, (5, 5), padding = 'valid')(block1)
    block2       = BatchNormalization(axis = 1)(block2)
    block2       = Activation('elu')(block2)
    block2       = Conv2D(Concat_Dense, (5, 5), padding = 'valid')(block2)
    block2       = BatchNormalization(axis = 1)(block2)
    block2       = Activation('elu')(block2)
    block2       = MaxPooling2D((2, 2))(block2)
    block2       = Dropout(dropoutRate)(block2)
        

    flatten2      = Flatten()(block2)
    flatten2_temp = Reshape((1, flatten2.shape[-1]))(flatten2)
    
    dense_model2        = Dense(2, kernel_constraint = max_norm(norm_rate))(flatten2)
    softmax2      = Activation('softmax')(dense_model2)

    model2 = Model(inputs=input2, outputs=softmax2)
    model2_Embedder = Model(inputs=input2, outputs=flatten2)
    
    softmax2_temp = Reshape((softmax2.shape[-1],1))(softmax2)
    
#%% Section 4: View-Net3: for processing Hand_engineered features #####################################################################################3###################################################################################3:
    Hand_engineered_features_num = 946
        
    input3   = Input(shape = (Hand_engineered_features_num,))

    block1       = BatchNormalization(axis = 1)(input3)
    block1       = Dense(Concat_Dense, input_shape = (Chans,))(block1)
    block1       = BatchNormalization(axis = 1)(block1)
    block1       = Activation('elu')(block1)
    block1       = Dropout(dropoutRate)(block1)
    

    flatten3      = Flatten()(block1)
    flatten3_temp = Reshape((1, flatten3.shape[-1]))(flatten3)
    
    dense_model3        = Dense(2, kernel_constraint = max_norm(norm_rate))(flatten3)
    softmax3      = Activation('softmax')(dense_model3)

    model3 = Model(inputs=input3, outputs=softmax3)
    model3_Embedder = Model(inputs=input3, outputs=flatten3)

    softmax3_temp = Reshape((softmax3.shape[-1],1))(softmax3)
    
#%% Section 5: Merged Models ########################################################################################################################################3 

    dense = Concatenate(axis=1)([flatten1_temp, flatten2_temp, flatten3_temp])  
    flatten4_1 = Attention_ViewSelector()(dense)
    
    dense = Concatenate(axis=1)([flatten1_temp, flatten3_temp, flatten2_temp])  
    flatten4_2 = Attention_ViewSelector()(dense)

    dense = Concatenate(axis=1)([flatten2_temp, flatten1_temp, flatten3_temp])  
    flatten4_3 = Attention_ViewSelector()(dense)

    dense = Concatenate(axis=1)([flatten2_temp, flatten3_temp, flatten1_temp])  
    flatten4_4 = Attention_ViewSelector()(dense)

    dense = Concatenate(axis=1)([flatten3_temp, flatten1_temp, flatten2_temp])  
    flatten4_5 = Attention_ViewSelector()(dense)

    dense = Concatenate(axis=1)([flatten3_temp, flatten2_temp, flatten1_temp])  
    flatten4_6 = Attention_ViewSelector()(dense)
    
    dense = Concatenate()([flatten4_1, flatten4_2, flatten4_3, flatten4_4, flatten4_5, flatten4_6])
    dense       = Dropout(dropoutRate)(dense)
    flatten4      = Flatten()(dense)

    dense        = Dense(2, kernel_constraint = max_norm(norm_rate))(flatten4)
    softmax      = Activation('softmax')(dense)
    model = Model(inputs=[input1, input2, input3], outputs=[softmax])

    plot_model(model, to_file=str(path_folder + 'model.png'), show_shapes=True)

    model_Embedder = Model(inputs=[input1, input2, input3], outputs=flatten4)
    
#%% Section 6: Using class weight to balance the classification problem : #######################################################################################################################33
    
    Class_weights = compute_class_weight('balanced', classes = np.unique(np.argmax(labels_train,1)),
                                         y = np.argmax(labels_train,1))
    class_weights = dict(enumerate(Class_weights)) 
       
#%% Section 7: compile and train the fAttNet model: ##################################################################################################################################
    model.compile(loss=['categorical_crossentropy'], optimizer='adam', 
              metrics = ['accuracy'])
    history = model.fit([View1_train, View2_train, View3_train], [labels_train], epochs=5, batch_size=32,shuffle=True,class_weight=class_weights)#,verbose=1,validation_split=0.15,callbacks=callbacks_list)
    
#%% Section 8: Save the trained fAttNet's weights:

#    model.save_weights(path_folder+'model_weights_sub'+str(Out_Subject)+'.h5')
    model.load_weights(path_folder+'model_weights_sub'+str(Out_Subject)+'.h5')
        
#%% Section 9: show the classification results and metrics (Confusion matrice and Acc, Sen, Spec, Prec and F1-Score):
    
    View3_test = np.squeeze(View3_test)
    
    labels_test_predict = model.predict([View1_test, View2_test, View3_test],batch_size=16)
            
    y_true = np.argmax(labels_test, 1)
    
    y_pred =  np.argmax(labels_test_predict, 1)
        
    C = confusion_matrix(y_true, y_pred, labels=[0,1]) # similar to this, produce confusion matrices of the ViewNet1-3
    
    print('\nSubject_out: ' + str(Out_Subject) + '/'+str(test_num)+' >>>>>>>>>>>> fAttNet model results: \n'+ str(C))
       
    print(classification_report(y_true, y_pred))

    
#%% Section 10: obtain embedded test data for visualizing the t-SNE and UMAP scatter plots:
    model1_Embedder_pred_reduced = model1_Embedder.predict(View1_test, batch_size=16)          
    model2_Embedder_pred_reduced = model2_Embedder.predict(View2_test, batch_size=16)            
    model3_Embedder_pred_reduced = model3_Embedder.predict(View3_test, batch_size=16)            
    model_Embedder_pred_reduced = model_Embedder.predict([View1_test, View2_test, View3_test], batch_size=16)

#%% Section 11: tSNE:
#    # get 2d embeddings
    tsne = manifold.TSNE(n_components=2)
    model1_Embedder_pred_reduced_tSNE = tsne.fit_transform(model1_Embedder_pred_reduced)
    
    tsne = manifold.TSNE(n_components=2)
    model2_Embedder_pred_reduced_tSNE = tsne.fit_transform(model2_Embedder_pred_reduced)

    tsne = manifold.TSNE(n_components=2)
    model3_Embedder_pred_reduced_tSNE = tsne.fit_transform(model3_Embedder_pred_reduced)
    
    tsne = manifold.TSNE(n_components=2)
    model_Embedder_pred_reduced_tSNE = tsne.fit_transform(model_Embedder_pred_reduced)

    # t-SNE subplot:
    color_scatter = []
    for i in range(len(y_true)):
        output_class = y_true[i]
        if(output_class == 0):
            color_scatter.append("#0000ff") # Blue ====> Class: 0
        else:
            color_scatter.append("#ff0000") # Red ====> Class: 1
                                       
    fig, axs = plt.subplots(2, 4)
        
    C = "confusion matrix of the View-Net1's prediction"
    acc = np.round(100*(C[0,0]+C[1,1])/(C[0,0]+C[1,1]+C[1,0]+C[0,1]))
    axs[1, 0].scatter(x = model1_Embedder_pred_reduced_tSNE[:, 0], 
                y=model1_Embedder_pred_reduced_tSNE[:, 1],
                color=color_scatter)
    axs[1, 0].set_title(str('View-Net1'))
    pop_a = mpatches.Patch(color='#0000ff', label='Non-seizure')
    pop_b = mpatches.Patch(color='#ff0000', label='Seizure')
    axs[1, 0].legend(handles=[pop_a,pop_b])
    axs[1, 0].set(xlabel='t-SNE 1', ylabel='t-SNE 2')
    np.save(str(path_folder + 'Sub'+str(Out_Subject)+'_model1_Embedder_pred_reduced_tSNE.npy'), model1_Embedder_pred_reduced_tSNE)
      
    C = "confusion matrix of the View-Net2's prediction"
    acc = np.round(100*(C[0,0]+C[1,1])/(C[0,0]+C[1,1]+C[1,0]+C[0,1]))
    axs[1, 1].scatter(x = model2_Embedder_pred_reduced_tSNE[:, 0], 
                y=model2_Embedder_pred_reduced_tSNE[:, 1],
                color=color_scatter)
    axs[1, 1].set_title(str('View-Net2'))
    axs[1, 1].legend(handles=[pop_a,pop_b])
    axs[1, 1].set(xlabel='t-SNE 1', ylabel='t-SNE 2')
    np.save(str(path_folder + 'Sub'+str(Out_Subject)+'_model2_Embedder_pred_reduced_tSNE.npy'), model2_Embedder_pred_reduced_tSNE)

    C = "confusion matrix of the View-Net3's prediction"
    acc = np.round(100*(C[0,0]+C[1,1])/(C[0,0]+C[1,1]+C[1,0]+C[0,1]))
    axs[1, 2].scatter(x = model3_Embedder_pred_reduced_tSNE[:, 0], 
                y=model3_Embedder_pred_reduced_tSNE[:, 1],
                color=color_scatter)
    axs[1, 2].set_title(str('View-Net3'))
    axs[1, 2].legend(handles=[pop_a,pop_b])
    axs[1, 2].set(xlabel='t-SNE 1', ylabel='t-SNE 2')
    np.save(str(path_folder + 'Sub'+str(Out_Subject)+'_model3_Embedder_pred_reduced_tSNE.npy'), model3_Embedder_pred_reduced_tSNE)

    C = "confusion matrix of the fAttNet's prediction"
    acc = np.round(100*(C[0,0]+C[1,1])/(C[0,0]+C[1,1]+C[1,0]+C[0,1]))
    axs[1, 3].scatter(x = model_Embedder_pred_reduced_tSNE[:, 0], 
                y=model_Embedder_pred_reduced_tSNE[:, 1],
                color=color_scatter)
    axs[1, 3].set_title(str('fAttNet'))
    axs[1, 3].legend(handles=[pop_a,pop_b])
    axs[1, 3].set(xlabel='t-SNE 1', ylabel='t-SNE 2')
    np.save(str(path_folder + 'Sub'+str(Out_Subject)+'_model_Embedder_pred_reduced_tSNE.npy'), model_Embedder_pred_reduced_tSNE)

#%% Section 12: UMAP: 
    dims = (int(flatten1.shape[1]),)
    n_components = 2
    encoder = Sequential([
       Input(input_shape=dims),
       Dense(units=n_components),
    ])
   
    encoder1 = encoder
    encoder2 = encoder
    encoder3 = encoder    
   
   
    embedder = ParametricUMAP(encoder=encoder1, dims=dims)    
    model1_Embedder_pred_reduced_UMAP = embedder.fit_transform(model1_Embedder_pred_reduced)
    
    dims = (int(flatten2.shape[1]),)
    n_components = 2
    encoder = Sequential([
       Input(input_shape=dims),
       Dense(units=n_components),
    ])
   
    encoder2 = encoder
    
    embedder = ParametricUMAP(encoder=encoder2, dims=dims)    
    model2_Embedder_pred_reduced_UMAP = embedder.fit_transform(model2_Embedder_pred_reduced)

    dims = (int(flatten3.shape[1]),)
    n_components = 2
    encoder = Sequential([
       Input(input_shape=dims),
       Dense(units=n_components),
    ])
   
    encoder3 = encoder
    
    embedder = ParametricUMAP(encoder=encoder3, dims=dims)    
    model3_Embedder_pred_reduced_UMAP = embedder.fit_transform(model3_Embedder_pred_reduced)

    dims = (int(flatten4.shape[1]),)
    n_components = 2
    encoder = Sequential([
       Input(input_shape=dims),
       Dense(units=n_components),
    ])
   
    encoder4 = encoder
    
    embedder = ParametricUMAP(encoder=encoder4, dims=dims)    
    model_Embedder_pred_reduced_UMAP = embedder.fit_transform(model_Embedder_pred_reduced)

## UMAP subplot:
    color_scatter = []
    for i in range(len(y_true)):
        output_class = y_true[i]
        if(output_class == 0):
            color_scatter.append("#0000ff") # Blue ====> Class: 0
        else:
            color_scatter.append("#ff0000") # Red ====> Class: 1
                                        

    C = "confusion matrix of the View-Net1's prediction"
    acc = np.round(100*(C[0,0]+C[1,1])/(C[0,0]+C[1,1]+C[1,0]+C[0,1]))
    axs[0, 0].scatter(x = model1_Embedder_pred_reduced_UMAP[:, 0], 
                y=model1_Embedder_pred_reduced_UMAP[:, 1],
                color=color_scatter)
    axs[0, 0].set_title(str('View-Net1'))
    axs[0, 0].legend(handles=[pop_a,pop_b])
    axs[0, 0].set(xlabel='UMAP 1', ylabel='UMAP 2')
    np.save(str(path_folder + 'Sub'+str(Out_Subject)+'_model1_Embedder_pred_reduced_UMAP.npy'), model1_Embedder_pred_reduced_UMAP)

    C = "confusion matrix of the View-Net2's prediction"
    acc = np.round(100*(C[0,0]+C[1,1])/(C[0,0]+C[1,1]+C[1,0]+C[0,1]))
    axs[0, 1].scatter(x = model2_Embedder_pred_reduced_UMAP[:, 0], 
                y=model2_Embedder_pred_reduced_UMAP[:, 1],
                color=color_scatter)
    axs[0, 1].set_title(str('View-Net2'))
    axs[0, 1].legend(handles=[pop_a,pop_b])
    axs[0, 1].set(xlabel='UMAP 1', ylabel='UMAP 2')
    np.save(str(path_folder + 'Sub'+str(Out_Subject)+'_model2_Embedder_pred_reduced_UMAP.npy'), model2_Embedder_pred_reduced_UMAP)

    C = "confusion matrix of the View-Net3's prediction"
    acc = np.round(100*(C[0,0]+C[1,1])/(C[0,0]+C[1,1]+C[1,0]+C[0,1]))
    axs[0, 2].scatter(x = model3_Embedder_pred_reduced_UMAP[:, 0], 
                y=model3_Embedder_pred_reduced_UMAP[:, 1],
                color=color_scatter)
    axs[0, 2].set_title(str('View-Net3'))
    axs[0, 2].legend(handles=[pop_a,pop_b])
    axs[0, 2].set(xlabel='UMAP 1', ylabel='UMAP 2')
    np.save(str(path_folder + 'Sub'+str(Out_Subject)+'_model3_Embedder_pred_reduced_UMAP.npy'), model3_Embedder_pred_reduced_UMAP)

    C = "confusion matrix of the fAttNet's prediction"
    acc = np.round(100*(C[0,0]+C[1,1])/(C[0,0]+C[1,1]+C[1,0]+C[0,1]))
    axs[0, 3].scatter(x = model_Embedder_pred_reduced_UMAP[:, 0], 
                y=model_Embedder_pred_reduced_UMAP[:, 1],
                color=color_scatter)
    axs[0, 3].set_title(str('fAttNet'))
    axs[0, 3].legend(handles=[pop_a,pop_b])
    axs[0, 3].set(xlabel='UMAP 1', ylabel='UMAP 2')
    np.save(str(path_folder + 'Sub'+str(Out_Subject)+'_model_Embedder_pred_reduced_UMAP.npy'), model_Embedder_pred_reduced_UMAP)

    # Hide x labels and tick labels for top plots and y ticks for right plots.
    for ax in axs.flat:
        ax.label_outer()

    fig.savefig(path_folder + 'Sub'+str(Out_Subject)+'.png')
    
#%% Section 13: Obtain saliency maps: ======>>>> requirements: install DeepExplain library (in the appendix files), Keras 2.2.4, Tensorflow 1.10.0 
    
    with DeepExplain(session=K.get_session()) as de:  # <-- init DeepExplain context
        # 1. Get the input tensor to the original model
        input_tensor = model1.layers[0].input
        
        # 2. We now target the output of the last dense layer (pre-softmax)
        # To do so, create a new model sharing the same layers untill the last dense (index -2)
        fModel = Model(inputs=input_tensor, outputs = model1.layers[-2].output)
        target_tensor = fModel(input_tensor)
        
        xs = View1_test
        ys = labels_test
        
        attributions_sal   = de.explain('saliency', target_tensor, input_tensor, xs=xs, ys=ys, batch_size=32)
        
        print ("Done") 
        
    np.save(path_folder+'Sub'+str(Out_Subject)+'_attributions_sal.npy', attributions_sal)        
